<a href="https://colab.research.google.com/github/LuisIrigoyen/trabajo_final_RL/blob/main/proyecto_final_APR_V05_en_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1.2. Localizar entorno de trabajo: Google colab o local

In [1]:
# ATENCIÓN!! Modificar ruta relativa a la práctica si es distinta (drive_root)
mount='/content/gdrive'
drive_root = mount + "/My Drive/08_MIAR/actividades/proyecto practico"

try:
  from google.colab import drive
  IN_COLAB=True
except:
  IN_COLAB=False


## 1.3. Montar carpeta de datos local (solo Colab)

In [2]:
# Switch to the directory on the Google Drive that you want to use
import os
if IN_COLAB:
  print("We're running Colab")

  if IN_COLAB:
    # Mount the Google Drive at mount
    print("Colab: mounting Google drive on ", mount)

    drive.mount(mount)

    # Create drive_root if it doesn't exist
    create_drive_root = True
    if create_drive_root:
      print("\nColab: making sure ", drive_root, " exists.")
      os.makedirs(drive_root, exist_ok=True)

    # Change to the directory
    print("\nColab: Changing directory to ", drive_root)
    %cd $drive_root
# Verify we're in the correct working directory
%pwd
print("Archivos en el directorio: ")
print(os.listdir())

We're running Colab
Colab: mounting Google drive on  /content/gdrive
Mounted at /content/gdrive

Colab: making sure  /content/gdrive/My Drive/08_MIAR/actividades/proyecto practico  exists.

Colab: Changing directory to  /content/gdrive/My Drive/08_MIAR/actividades/proyecto practico
/content/gdrive/My Drive/08_MIAR/actividades/proyecto practico
Archivos en el directorio: 
['dqn_models']


## 1.4. Instalar librerías necesarias

In [2]:
if IN_COLAB:
  %pip install gym==0.17.3
  %pip install git+https://github.com/Kojoley/atari-py.git
  %pip install keras-rl2==1.0.5
  %pip install tensorflow==2.12  #2.8
else:
  %pip install gym==0.17.3
  %pip install git+https://github.com/Kojoley/atari-py.git
  %pip install pyglet==1.5.0
  %pip install h5py==3.1.0
  %pip install Pillow==9.5.0
  %pip install keras-rl2==1.0.5
  %pip install Keras==2.2.4
  %pip install tensorflow==2.5.3
  %pip install torch==2.0.1
  %pip install agents==1.4.0

  Cloning https://github.com/Kojoley/atari-py.git to /tmp/pip-req-build-rak2yilz
  Running command git clone --filter=blob:none --quiet https://github.com/Kojoley/atari-py.git /tmp/pip-req-build-rak2yilz
  Resolved https://github.com/Kojoley/atari-py.git to commit 86a1e05c0a95e9e6233c3a413521fdb34ca8a089
  Preparing metadata (setup.py) ... done


## **PARTE 3**. Desarrollo y preguntas
Importar librerías

In [3]:
from __future__ import division

from PIL import Image
import numpy as np
import gym

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Convolution2D, Permute, BatchNormalization
#from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K

from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from rl.callbacks import FileLogger, ModelIntervalCheckpoint

#### Configuración base

In [4]:
INPUT_SHAPE = (84, 84)
WINDOW_LENGTH = 4

env_name = 'SpaceInvaders-v0'
env = gym.make(env_name, frameskip=1)

np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n

#--- nuevo -- ejfm
height, width, channels = env.observation_space.shape
obs = env.reset()
print(height, width, channels)
print("Forma de la observación:", obs.shape)  # Debe ser (height, width, channels)
env.unwrapped.get_action_meanings()

210 160 3
Forma de la observación: (210, 160, 3)


['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [5]:
class AtariProcessor(Processor):
    def process_observation(self, observation):
        assert observation.ndim == 3  # (height, width, channel)
        img = Image.fromarray(observation)
        img = img.resize(INPUT_SHAPE).convert('L')
        processed_observation = np.array(img)
        assert processed_observation.shape == INPUT_SHAPE
        return processed_observation.astype('uint8')

    def process_state_batch(self, batch):

        # ejfm -- modificado
        # Agrega la dimensión de canales (axis=-1) si no está presente
        if batch.ndim == 4:                         # Forma: (batch_size, window_length, h, w)
            batch = np.expand_dims(batch, axis=-1)  # Nueva forma: (batch_size, window_length, h, w, 1)
            processed_batch = batch.astype('float32') / 255.
        return processed_batch

    def process_reward(self, reward):
        return np.clip(reward, -1., 1.)

## 1. EJFM Implementación de la red neuronal

In [6]:
def build_model(height, width, channels, actions):
    model = Sequential()

    # Capa 1: Conv2D con BatchNorm y ReLU
    model.add(Convolution2D(32, (8, 8), strides=(4, 4), padding='same', input_shape=(4,84,84,1)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    # Capa 2: Conv2D con BatchNorm y ReLU
    model.add(Convolution2D(64, (4, 4), strides=(2, 2), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    # Capa 3: Conv2D con BatchNorm y ReLU
    model.add(Convolution2D(64, (3, 3), strides=(1, 1), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    # Aplanar + Capas Dense
    model.add(Flatten())

    # Capa Fully Connected con Dropout
    model.add(Dense(512))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    # model.add(Dropout(0.3))  # Opcional para evitar overfitting

    # Capa de salida
    model.add(Dense(actions, activation='linear'))

    return model

In [7]:
model = build_model(84, 84, 1, nb_actions)
model.summary()

Instructions for updating:
Colocations handled automatically by placer.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 4, 21, 21, 32)     2080      
                                                                 
 batch_normalization (BatchN  (None, 4, 21, 21, 32)    128       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 4, 21, 21, 32)     0         
                                                                 
 conv2d_1 (Conv2D)           (None, 4, 11, 11, 64)     32832     
                                                                 
 batch_normalization_1 (Batc  (None, 4, 11, 11, 64)    256       
 hNormalization)                                                 
                                                                 
 activation_1 (Activation)   (None, 4, 11, 11, 64)     0

## 2. EJFM Implementación de la solución DQN

In [8]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [17]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=0.02, value_test=0.005, nb_steps=750000)
    memory = SequentialMemory(limit=500000, window_length=4)
    dqn = DQNAgent(model=model,
                   memory=memory,
                   policy=policy,
                   enable_double_dqn=True,
                   enable_dueling_network=True,
                   dueling_type='avg',
                   nb_actions=nb_actions,
                   batch_size=64,
                   target_model_update=5000,
                   processor=AtariProcessor()
                  )
    return dqn

In [21]:
from rl.callbacks import ModelIntervalCheckpoint

# Guardar el modelo cada 50,000 pasos
checkpoint_callback = ModelIntervalCheckpoint(
    filepath='checkpoints/dqn_v05_weights_{step}.h5f',
    interval=50000,  # cada 50,000 pasos
    verbose=1
)

In [22]:
import os
os.makedirs("checkpoints", exist_ok=True)

In [23]:
from keras.optimizers import Adam

dqn = build_agent(model, nb_actions)
dqn.compile(Adam(learning_rate=0.00025, clipnorm=1.0))


In [31]:
from datetime import datetime

print(f"⏱ Tiempo de inicio: {datetime.now().strftime('%H:%M:%S')}")

dqn.fit(env, nb_steps=50000, visualize=False, verbose=2,callbacks=[checkpoint_callback])

print(f"⏱ Tiempo de fin: {datetime.now().strftime('%H:%M:%S')}")

# Guardando modelo
dqn.save_weights('dqn_05_weights.h5f')


⏱ Tiempo de inicio: 01:43:22
Training for 50000 steps ...


/usr/local/lib/python3.11/dist-packages/keras/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


  1963/50000: episode: 1, duration: 298.547s, episode steps: 1963, steps per second:   7, episode reward:  7.000, mean reward:  0.004 [ 0.000,  1.000], mean action: 2.424 [0.000, 5.000],  loss: 0.055924, mean_q: 0.142100, mean_eps: 0.998064
  3626/50000: episode: 2, duration: 517.496s, episode steps: 1663, steps per second:   3, episode reward:  7.000, mean reward:  0.004 [ 0.000,  1.000], mean action: 2.471 [0.000, 5.000],  loss: 0.002520, mean_q: 0.059741, mean_eps: 0.996349
  6277/50000: episode: 3, duration: 842.066s, episode steps: 2651, steps per second:   3, episode reward: 12.000, mean reward:  0.005 [ 0.000,  1.000], mean action: 2.472 [0.000, 5.000],  loss: 0.003159, mean_q: 0.068677, mean_eps: 0.993531
  8636/50000: episode: 4, duration: 744.825s, episode steps: 2359, steps per second:   3, episode reward: 12.000, mean reward:  0.005 [ 0.000,  1.000], mean action: 2.479 [0.000, 5.000],  loss: 0.003015, mean_q: 0.069601, mean_eps: 0.990257
 10617/50000: episode: 5, duration: 

In [32]:
# Testing part to calculate the mean reward
#weights_filename = 'dqn_{}_weights.h5f'.format(env_name)
weights_filename = 'dqn_05_weights.h5f'.format(env_name)
dqn.load_weights(weights_filename)
dqn.test(env, nb_episodes=10, visualize=False)

Testing for 10 episodes ...
Episode 1: reward: 5.000, steps: 2047
Episode 2: reward: 4.000, steps: 2049
Episode 3: reward: 3.000, steps: 2033
Episode 4: reward: 4.000, steps: 2015
Episode 5: reward: 8.000, steps: 2839
Episode 6: reward: 3.000, steps: 2099
Episode 7: reward: 5.000, steps: 1921
Episode 8: reward: 6.000, steps: 2749
Episode 9: reward: 4.000, steps: 2057
Episode 10: reward: 4.000, steps: 2753


In [33]:
from google.colab import drive
import shutil

# 1. Montar Google Drive
#drive.mount('/content/drive')

# 2. Copiar archivo (ej: modelo entrenado)
shutil.copy(
    'dqn_05_weights.h5f',  # Ruta en Colab
    '/content/drive/MyDrive/Colab_Outputs/dqn_05_weights.h5f'  # Ruta en Drive
)

print("¡Archivo copiado a Drive!")

FileNotFoundError: [Errno 2] No such file or directory: 'dqn_05_weights.h5f'